In [ ]:
from tracktable.core import data_directory
from tracktable.core import geomath as geomath
import os.path
import matplotlib.pyplot as plt
from tracktable.applications.assemble_trajectories import AssembleTrajectoryFromPoints
from tracktable.domain.terrestrial import TrajectoryPointReader
from datetime import timedelta
from tracktable.render.render_trajectories import render_trajectories, render_trajectories_separate

import numpy as np
from datetime import datetime

import pandas as pd

from tracktable.domain import terrestrial
from tracktable.render.mapmaker import mapmaker

import tracktable.domain.terrestrial
from tracktable.lib._domain_algorithm_overloads import end_to_end_distance as _end_to_end_distance
from tracktable.lib._domain_algorithm_overloads import length as _length
from tracktable.lib._domain_algorithm_overloads import speed_between as _speed_between

In [ ]:
def load_trajectories_from_delim_txt(filename):
    reader = TrajectoryPointReader()
    reader.input = open(filename, "r")
    reader.field_delimiter = '\t'
    reader.object_id_column = 0
    reader.timestamp_column = 1
    reader.coordinates[0] = 2     #longitude
    reader.coordinates[1] = 3     #latitude
    reader.set_real_field_column('heading', 5)
    reader.set_real_field_column('altitude', 6)
    reader.set_time_field_column('expected_departure_time', 21)
    reader.set_time_field_column('expected_arrival_time', 22)
    reader.set_time_field_column('actual_departure_time', 23)
    reader.set_time_field_column('actual_arrival_time', 24)
    builder = AssembleTrajectoryFromPoints()
    builder.input = reader
    builder.separation_distance = 100 #km
    builder.separation_time = timedelta(minutes = 20)
    builder.minimum_length = 5 #points
    trajectories = list(builder)
    return trajectories

In [ ]:
# tracktable.domain.terrestrial.TrajectoryWriter

# tracktable.domain.terrestrial.TrajectoryWriter(output)

In [ ]:
def max_altitude(trajectory):
    #altitude = point.properties["altitude"] -- .append to make a list
    all_altitude_values = list()
    for point in trajectory:
        if "altitude" in point.properties:
            altitude = point.properties["altitude"]
            if altitude is not None: 
                all_altitude_values.append(altitude)
        
    #alt_list = [point.properties["altitude"] for point in trajectory]  # --> list comprehension 
    max_alt = max(all_altitude_values)
    return max_alt

# max_alt_list = [max_altitude(trajectory) for trajectory in load_trajectories]
# plt.hist(max_alt_list)

In [ ]:
def maximum_speed(trajectory):
    speed_list = list()
    for point in trajectory:
        if "speed" in point.properties:
            speed = point.properties["speed"]
            if speed is not None: 
                speed_list.append(speed)
        
    max_speed = max(speed_list)
    return max_speed

In [ ]:
def end_to_end_distance(traj):
    """Return the distance between a path's endpoints

    This is just the crow-flight distance between start and end points rather
    than the total distance traveled.

    Domain Information:

      Terrestrial: distance in km

      Cartesian2D: distance in units

      Cartesian3D: distance in units

    Args:
      trajectory (Trajectory): Path whose length we want

    Returns:
      Length in domain-dependent units

    """
    
    return _end_to_end_distance(traj)

In [ ]:
def length(traj):
    """Return the length of a path in domain-dependent units

    This is the total length of all segments in the trajectory.

    Domain Information:

      Terrestrial: distance in km

      Cartesian2D: distance in units

      Cartesian3D: distance in units

    Args:
      trajectory (Trajectory): Path whose length we want

    Returns:
      Length in domain-dependent units

    """

    return _length(traj)

In [ ]:
def length_and_end_to_end_distance_ratio(traj):
    """Return the ratio of end-to-end distance to length of a path

    This function calculates both the total length of all segments in the trajectory
    and the crow-flight distance between the start and end points. Then, it returns
    the ratio of the end-to-end distance to the total length.

    Domain Information:

      Terrestrial: distance in km

      Cartesian2D: distance in units

      Cartesian3D: distance in units

    Args:
      trajectory (Trajectory): Path whose ratio of end-to-end distance to length we want

    Returns:
      float: Ratio of end-to-end distance to length
    """
    total_length = _length(traj)
    end_to_end_dist = _end_to_end_distance(traj)
    return end_to_end_dist / total_length

In [ ]:
def afltduration(traj):
    afllist = list()
    actdeplist = list()
    actarrlist = list()
    for point in traj:
        if "expected_departure_time" in point.properties:
            expdeparturetime = point.properties["actual_arrival_time"]
            if expdeparturetime is not None: 
                actarrlist.append(expdeparturetime)
        
        if "actual_departure_time" in point.properties:
            actdeparturetime = point.properties["actual_departure_time"]
            if actdeparturetime is not None: 
                actdeplist.append(actdeparturetime)
    
    for departure, arrival in zip(actdeplist, actarrlist):
        if departure and arrival:  
            flight_duration = arrival - departure
            afllist.append(flight_duration.total_seconds() / -60)  # Convert to minutes
    return afllist

In [ ]:
def altitude_list(traj):
    all_altitude_values = list()
    for point in traj:
        if "altitude" in point.properties:
            altitude = point.properties["altitude"]
            if altitude is not None: 
                all_altitude_values.append(altitude)
        
    return all_altitude_values

In [ ]:
def recompute_speed(traj, target_attribute_name="speed"):
    """Use points and timestamps to compute speed

    The speed data in trajectories is often suspect. This method goes
    through and recomputes it based on the distance between
    neighboring points and the time elapsed between those points.

    The speed at point N is computed using the distance and time since
    point N-1. The speed at point 0 is copied from point 1.

    Args:
      trajectory (Trajectory): Any Tracktable trajectory

    Keyword Arguments:
      target_attribute_name (str): Speed will be stored in this property at
          each point. Defaults to 'speed'. (Default: "speed")

    The trajectory will be modified in place instead of returning a
    new copy.
    """

    if len(traj) == 0:
        return []
    elif len(traj) == 1:
        traj[0].properties[target_attribute_name] = 0
        return [0]
    else:
        speeds = [None] * len(traj) 
        for point_index in range(1, len(traj)):
            speed = _speed_between(traj[point_index - 1], traj[point_index])
            traj[point_index].properties[target_attribute_name] = speed
            speeds[point_index] = speed 
        speeds[0] = speeds[1] 
        traj[0].properties[target_attribute_name] = speeds[0]
        return speeds

In [ ]:
# f = plt.figure(size=(8, 6), dpi=100)

# (mymap, initial_artists) = mapmaker('australia',
#                                     draw_coastlines=True,
#                                     draw_countries=False,
#                                     draw_states=False,
#                                     draw_lonlat=True,
#                                     lonlat_spacing=2,
#                                     lonlat_linewidth=0.5)

In [ ]:


# Assuming your data is stored in a variable named 'data'
data = """object_id timestamp longitude latitude speed heading altitude callsign flight_number unknown status unknown unknown tail_number aircraft_type unknown unknown unknown unknown unknown unknown unknown expected_departure_time expected_arrival_time actual_departure_time actual_arrival_time origin_icao origin_name origin_iata origin_longitude origin_latitude destination_icao destination_name destination_iata destination_longitude destination_latitude unknown unknown unknown unknown unknown unknown route_icao unknown route_with_names route_iata
UAL239 2015-01-02 00:00:00 -102.574 38.2258 79 31000 UAL239 UA239 2015-01-01 17:05:00 2015-01-01 10:43:00 2015-01-01 13:26:00 KLAX LAX (Los Angeles) LAX -118.408 33.9425 KSFO SFO (San Francisco) SFO -122.375 37.619 808.382 808.382 293.101 KLAX-KSFO LAX (Los Angeles)-SFO (San Francisco) LAX-SFO
SKW6507 2015-01-02 00:00:00 -103.496 41.4022 181 30000 SKW6507 YT6507 2015-01-01 17:37:00 2015-01-01 16:21:00 2015-01-01 16:17:00 CYQR YQR (Regina) YQR -104.666 50.4319 KDEN DEN (Denver) DEN -104.673 39.8617 543.959 543.959 634.212 CYQR-KDEN YQR (Regina)-DEN (Denver) YQR-DEN
DAL1914 2015-01-02 00:00:00 -103.573 40.3697 75 21300 DAL1914 DL1914 A320 2015-01-01 17:22:00 2015-01-01 14:40:00 2015-01-02 00:07:30 2015-01-01 19:53:00 KLGA LGA (New York) LGA -73.8726 40.7772 KDEN DEN (Denver) DEN -104.673 39.8617 1347.27 1347.27 1402.84 KLGA-KDEN LGA (New York)-DEN (Denver) LGA-DEN"""

# Split the data into rows
rows = data.split('\n')

# Extract the header row
header_row = rows[0].split()

# Search for the column name 'expected_departure_time'
expected_departure_time_index = header_row.index('actual_departure_time')

# Column number is one-based, so add 1 to the index
column_number = expected_departure_time_index + 1

print("Column number assigned to 'expected_departure_time':", column_number)

In [ ]:
load_trajectories = load_trajectories_from_delim_txt("asdi_2015_01_02.tsv")

In [ ]:
trajectory = load_trajectories[0:29057]

In [ ]:
# render_trajectories(trajectory)
# render_trajectories(load_trajectories[0:100])

In [ ]:
#tracktable.core.geomath.recompute_speed(trajectory, target_attribute_name="speed")
#tracktable.core.geomath.length(trajectory)
#tracktable.core.geomath.end_to_end_distance(trajectory)

In [ ]:
endtoend_list = list()
for traj in trajectory:
    eedistance = end_to_end_distance(traj)
    if eedistance >= 0 and eedistance is not None:
        endtoend_list.append(eedistance)
length_of_list = len(endtoend_list)
print("Length of endtoend_list:", length_of_list)

In [ ]:
arclength_list = list()
for traj in trajectory:
    alength = length(traj)
    if alength >= 0 and alength is not None:
        arclength_list.append(alength)
length_of_list = len(arclength_list)
print("Length of arclength_list:", length_of_list)

In [ ]:
#altlist = altitude_list(trajectory)
#print(altlist)
altlists = []
for traj in trajectory:
    altitudes = altitude_list(traj) 
    traj_altitudes = []  
    for altitude in altitudes:  
        if altitude >= 0 and altitude is not None:
            traj_altitudes.append(altitude)
    altlists.append(traj_altitudes) 
print(altlists)

In [ ]:
speed_lists = [] 
for traj in trajectory:
    speeds = recompute_speed(traj, target_attribute_name="speed")  
    traj_speeds = [] 
    for speed in speeds: 
        if speed >= 0 and speed is not None:
            traj_speeds.append(speed)
    speed_lists.append(traj_speeds) 
print(speed_lists)

In [ ]:
altitude_differences_lists = []
speed_differences_lists = []
for traj in trajectory:
    #Compute speed and altitude lists for the current trajectory
    speeds = recompute_speed(traj)
    altitudes = altitude_list(traj)
    #Calculate the differences between speed values
    speed_differences = [speeds[i] - speeds[i-1] for i in range(1, len(speeds))]
    speed_differences_lists.append(speed_differences)
    #Calculate the differences between altitude values
    altitude_differences = [altitudes[i] - altitudes[i-1] for i in range(1, len(altitudes))]
    altitude_differences_lists.append(altitude_differences)
    print("Speed Differences:", speed_differences)
    print("Altitude Differences:", altitude_differences)


In [ ]:
#Initialize counters
speed_sign_change_count = 0
altitude_sign_change_count = 0
both_sign_change_count = 0
for i in range(len(speed_differences_lists)):
    #Check for sign changes in speed differences
    if any((j < len(speed_differences_lists[i]) - 1 and
            ((diff > 0 and speed_differences_lists[i][j+1] < 0) or 
             (diff < 0 and speed_differences_lists[i][j+1] > 0)))
           for j, diff in enumerate(speed_differences_lists[i])):
        speed_sign_change_count += 1
    #Check for sign changes in altitude differences
    if any((j < len(altitude_differences_lists[i]) - 1 and
            ((diff > 0 and altitude_differences_lists[i][j+1] < 0) or 
             (diff < 0 and altitude_differences_lists[i][j+1] > 0)))
           for j, diff in enumerate(altitude_differences_lists[i])):
        altitude_sign_change_count += 1
    #Check for sign changes in both speed and altitude differences
    if any((j < len(speed_differences_lists[i]) - 1 and
            j < len(altitude_differences_lists[i]) - 1 and
            (((diff_speed > 0 and speed_differences_lists[i][j+1] < 0) or 
              (diff_speed < 0 and speed_differences_lists[i][j+1] > 0)) and 
             ((diff_altitude > 0 and altitude_differences_lists[i][j+1] < 0) or 
              (diff_altitude < 0 and altitude_differences_lists[i][j+1] > 0))))
           for j, (diff_speed, diff_altitude) in enumerate(zip(speed_differences_lists[i], altitude_differences_lists[i]))):
        both_sign_change_count += 1
print("Number of trajectories with a sign change in speed:", speed_sign_change_count)
print("Number of trajectories with a sign change in altitude:", altitude_sign_change_count)
print("Number of trajectories with a sign change in both speed and altitude:", both_sign_change_count)





In [ ]:
#eearclength_ratio = [a / e for a, e in zip(endtoend_list, arclength_list) if a != 0 and e != 0 and a / e <= .99]
#length_of_list = len(eearclength_ratio)
#print("Length of eearclength_ratio:", length_of_list)

In [ ]:
#afldurationlist = list()
#for traj in trajectory:
    #afldur = afltduration(traj)
    #afldurationlist.append(afldur)

In [ ]:
#altitude = max_altitude(trajectory)
#speed = maximum_speed(trajectory)
# endtoenddistance = endtoend_list
# arclength = arclength_list
#actualflightduration = afldurationlist

In [ ]:
#print(altitude)
#print(speed)
#print(endtoenddistance)
#print(arclength)
#print(eearclength_ratio)
#print(actualflightduration)
#print(afltdurationlist)
#ratioeedistancealength = (endtoenddistance/arclength)
#print(ratioeedistancealength)


In [ ]:
#def current_time_frac(trajectory):
    #for point in trajectory:
        #time = geomath.current_time_fraction(point)
        #print(time)

In [ ]:
#for point in trajectory:
    #time = geomath.current_time_fraction(point)
    #print(time)

In [ ]:
#max_speed_list = [maximum_speed(trajectory) for trajectory in load_trajectories]

In [ ]:
# threshold = 0.8697229263968138
# eearclength_ratio_array = np.array(eearclength_ratio)
# Q1 = np.percentile(eearclength_ratio_array, 25)
# Q3 = np.percentile(eearclength_ratio_array, 75)
# IQR = Q3 - Q1
# lower_bound = Q1 - 1.5 * IQR
# upper_bound = Q3 + 1.5 * IQR
# #print(IQR)
# print(upper_bound)
# print(lower_bound)
# #outliers = [value for value in eearclength_ratio if value < lower_bound or value > upper_bound]
# #print("Outliers:", outliers)
# below_threshold = np.sum(eearclength_ratio_array < threshold)
# percent_below_threshold = (below_threshold / len(eearclength_ratio)) * 100
# print(f"{percent_below_threshold}% of values are below {threshold}.")
# print(below_threshold)

In [ ]:
#plt.hist(max_speed_list)
#plt.hist(endtoenddistance)
#plt.hist(arclength)
#plt.hist(eearclength_ratio, bins = 100)
#plt.xlim(0, 1)  # Set the x-axis limits
#plt.show()

In [ ]:
condition = .72
num_bins = 72
total_trajectories = len(endtoend_list)
while condition >= 0.70:
    # Apply condition and calculate eearclength_ratio
    filtered_indices = [i for i, (a, e) in enumerate(zip(endtoend_list, arclength_list)) if a != 0 and e != 0 and a / e <= condition]
    eearclength_ratio = [a / e for a, e in zip(endtoend_list, arclength_list) if a != 0 and e != 0 and a / e <= condition]
    # Filter trajectories based on the condition
    filtered_trajectory = [trajectory[i] for i in filtered_indices]
    # Calculate lower bound threshold for outliers
    q1 = np.percentile(eearclength_ratio, 25)
    q3 = np.percentile(eearclength_ratio, 75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    # Filter outlier trajectories based on the lower bound threshold
    outlier_indices = [i for i, ratio in enumerate(eearclength_ratio) if ratio < lower_bound]
    outlier_trajectories = [filtered_trajectory[i] for i in outlier_indices]
    render_trajectories(outlier_trajectories)
    display(render_trajectories(outlier_trajectories))
    plt.hist(eearclength_ratio, bins=num_bins)
    plt.title(f'End to End Distance/Arc Length <= {condition:.2f}')
    plt.ylabel('Frequency')
    num_below_threshold = np.sum(np.array(eearclength_ratio) < lower_bound)
    percent_below_threshold = (num_below_threshold / 29046) * 100
    percent_of_orig_dataset_left = (len(eearclength_ratio) / 29046) * 100
    plt.axvline(lower_bound, color='r', linestyle='--', label=f'Outlier Threshold: {lower_bound:.2f}')
    plt.legend(loc='upper left')
    plt.text(0.01, 0.78, f'# Below Threshold: {num_below_threshold}\nPercent of Original Dataset Below Threshold: {percent_below_threshold:.2f}% \nPercent Of Original Dataset Remaining: {percent_of_orig_dataset_left:.1f}%', transform=plt.gca().transAxes, fontsize=10, color='black')
    plt.show()
    condition -= 0.01
    num_bins -= 1


In [ ]:
# condition = .90  # Initial condition value
# # num_bins = 100  # Uncomment if you want to use num_bins
# total_trajectories = len(endtoend_list)

# # Apply condition and calculate eearclength_ratio
# eearclength_ratio = [a / e for a, e in zip(endtoend_list, arclength_list) if a != 0 and e != 0 and a / e <= condition]
# # Calculate lower bound threshold for outliers
# q1 = np.percentile(eearclength_ratio, 25)
# q3 = np.percentile(eearclength_ratio, 75)
# iqr = q3 - q1
# lower_bound = q1 - 1.5 * iqr
# # Filter trajectories based on the condition
# outlier_trajectories = [trajectory[i] for i, ratio in enumerate(eearclength_ratio) if ratio < lower_bound]
# print(len(outlier_trajectories))
# render_trajectories(outlier_trajectories)
# # Plot histogram
# # plt.hist(eearclength_ratio, bins=num_bins)
# # plt.title(f'End to End Distance/Arc Length <= {condition:.2f}')
# # plt.ylabel('Frequency')
# # Calculate number of trajectories below threshold
# # num_below_threshold = np.sum(np.array(eearclength_ratio) < lower_bound)
# # Calculate percentage of trajectories below threshold
# # percent_below_threshold = (num_below_threshold / 29046) * 100
# # percent_of_orig_dataset_left = (len(eearclength_ratio) / 29046) * 100
# # plt.axvline(lower_bound, color='r', linestyle='--', label=f'Outlier Threshold: {lower_bound:.2f}')
# # plt.legend(loc='upper left')
# # plt.text(0.01, 0.78, f'# Below Threshold: {num_below_threshold}\nPercent of Original Dataset Below Threshold: {percent_below_threshold:.2f}% \nPercent Of Original Dataset Remaining: {percent_of_orig_dataset_left:.1f}%', transform=plt.gca().transAxes, fontsize=10, color='black')
# # plt.show()


In [ ]:
# speed_list = list()
# for point in trajectory:
#     if "speed" in point.properties:
#         speed = point.properties["speed"]
#         print(speed)
#         if speed is not None: 
#             speed_list.append(speed)

In [ ]:
# print(speed)

In [ ]:
# To Do:
#     speed_between

# sanity_check_distance_less_than
# current_length